In [6]:
import sys
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.font_manager import FontProperties
sys.path.append(os.path.abspath("../code_base"))
from _info_ import ear_types, degrees, acupoints_name, cm
from _common_ import angles, split_xy_xyv
from localization_error import errors_csv
# from ..code_base._info_ import ear_types, degrees, acupoints_name, cm
# from ..code_base._common_ import angles, split_xy_xyv
# from ..code_base.localization_error import error_csv

In [27]:
class occlusion_errors_csv(errors_csv):
    def __init__(self):
        super().__init__("occluded_result")
        self.occluded_img_csv = "../occluded_imgs_csv"
    
        
    def generate(self):
        for ear_type in ear_types:
            names = os.listdir(os.path.join("..", self.data_folder, ear_type))
            for name in names:
                for deg in degrees:
                    gt = os.path.join(self.gt_folder, ear_type, name , deg, "gt.csv")
                    gt_df = self.read_csv(gt)
        
                    
                    indexes = os.listdir(os.path.join(self.occluded_img_csv, ear_type, name, deg))
                    indexes = sorted(indexes, key = lambda index : int(index))
                    df = []
                    for index in indexes:
                        
                        pred = os.path.join("..", self.data_folder, ear_type, name, "pred", deg, index, "pred.csv")
                        pred_df = self.read_csv(pred)
                    
                        
                        scale0_x, scale0_y, scale0_v= split_xy_xyv(gt_df.iloc[int(index), 0])
                        scale11_x, scale11_y, scale11_v = split_xy_xyv(gt_df.iloc[int(index), 11]) 
                        scale = cm[name] / dist((scale0_x, scale0_y), (scale11_x, scale11_y))
                        
                        invisible = os.path.join(self.occluded_img_csv, ear_type, name, deg, index, "invisible", "invisible.csv")
                        invisible_df = self.read_csv(invisible)
            
                        
                        error_df = pd.DataFrame(columns=pred_df.columns, index = pred_df.index)
                        for i in range(0, len(pred_df)):
                            
                            for j in range(0, len(pred_df.columns)):
                                gt_x, gt_y, gt_v = split_xy_xyv(gt_df.iloc[int(index),j])
              
                                if gt_v == "2" and invisible_df.iloc[i,j] == 1:
                                    pred_x, pred_y = split_xy_xyv(pred_df.iloc[i,j])
                                    error_df.iloc[i,j] = scale * dist((gt_x, gt_y), (pred_x, pred_y))
                        
                        error_np = error_df.to_numpy()
                        error_np = np.nanmean(error_np.astype('float64'), axis = 0)
                        df.append(error_np)
                    df = pd.DataFrame(data = df)


                    error_path = os.path.join("..", self.data_folder, ear_type, name, "error", deg)
                    if not os.path.isdir(error_path):
                        os.makedirs(error_path)
                    df.to_csv(os.path.join(error_path, "error.csv"))
                    
                    
                        

In [28]:
A = occlusion_errors_csv()
A.generate()

C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\user\AppData\Local\Temp\ipykernel_12796\4084963029.py:44: RuntimeWarning: Mean of empty slice
  error_np = np.nanmean(error_np.astype('float64'), axis = 0)
C:\Users\u